# Summary Forecast and Receipts

This notebook provides a summary roll-up of retail dollars of demand by Category and Collection for 2025 and 2026.

## Setup and Imports

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options for better formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Load Data Files

In [2]:
import os

# Get the directory where this notebook is located
base_path = os.path.dirname(os.path.abspath('__file__'))
data_path = os.path.join(base_path, '..', 'data')

# Load demand forecast file
demand_forecast = pd.read_csv(os.path.join(data_path, 'demand_forecast_file.csv'))
print(f"Demand forecast loaded: {len(demand_forecast):,} rows")

# Load catalog data
catalog = pd.read_csv(os.path.join(data_path, 'cz_catalog_data.csv'))
print(f"Catalog loaded: {len(catalog):,} rows")

# Display sample data
print("\nDemand Forecast columns:", demand_forecast.columns.tolist())
print("\nCatalog columns:", catalog.columns.tolist())

Demand forecast loaded: 293,232 rows
Catalog loaded: 6,109 rows

Demand Forecast columns: ['SKU', 'MONTH', 'UNIT DEMAND', 'UNIT SALES']

Catalog columns: ['SKU', 'SKU_DESCRIPTION', 'CATEGORY', 'SUB_CATEGORY', 'COLOR_NAME', 'COLLECTION', 'SIZE', 'PLANNING_CATEGORY', 'LAUNCH_YEAR_SEASON', 'LAUNCH_DATE', 'PLANNING_STATUS', 'ASSORTMENT_STATUS', 'FULL_PRICE_RETAIL', 'VENDOR_NAME', 'VENDOR_COST', 'VENDOR_COST_USD', 'ITEM_MOQS', 'TRANSIT_TIME', 'PRODUCTION_TIME', 'TOTAL_LEAD_TIME', 'IS_MTO', 'IS_DROPSHIP']


## Data Preparation

In [3]:
# Clean and prepare data
demand_forecast['MONTH'] = pd.to_datetime(demand_forecast['MONTH'])
demand_forecast['SKU'] = demand_forecast['SKU'].astype(str)
demand_forecast['UNIT DEMAND'] = pd.to_numeric(demand_forecast['UNIT DEMAND'], errors='coerce').fillna(0)

# Clean catalog data
catalog['SKU'] = catalog['SKU'].astype(str)
catalog['FULL_PRICE_RETAIL'] = pd.to_numeric(catalog['FULL_PRICE_RETAIL'], errors='coerce').fillna(0)
catalog['CATEGORY'] = catalog['CATEGORY'].fillna('UNKNOWN').astype(str)
catalog['COLLECTION'] = catalog['COLLECTION'].fillna('UNKNOWN').astype(str)

# Filter to 2025 and 2026 only
demand_forecast['YEAR'] = demand_forecast['MONTH'].dt.year
demand_forecast_filtered = demand_forecast[demand_forecast['YEAR'].isin([2025, 2026])].copy()
print(f"Filtered to 2025-2026: {len(demand_forecast_filtered):,} rows")

# Create month name column for display
demand_forecast_filtered['MONTH_NAME'] = demand_forecast_filtered['MONTH'].dt.strftime('%b')
demand_forecast_filtered['YEAR_MONTH'] = demand_forecast_filtered['MONTH'].dt.strftime('%Y-%m')

Filtered to 2025-2026: 146,616 rows


In [4]:
# Merge demand forecast with catalog to get CATEGORY, COLLECTION, and FULL_PRICE_RETAIL
merged_data = demand_forecast_filtered.merge(
    catalog[['SKU', 'CATEGORY', 'COLLECTION', 'FULL_PRICE_RETAIL']],
    on='SKU',
    how='left'
)

# Fill missing values
merged_data['CATEGORY'] = merged_data['CATEGORY'].fillna('UNKNOWN')
merged_data['COLLECTION'] = merged_data['COLLECTION'].fillna('UNKNOWN')
merged_data['FULL_PRICE_RETAIL'] = merged_data['FULL_PRICE_RETAIL'].fillna(0)

# Calculate retail dollars of demand
merged_data['RETAIL_DEMAND_DOLLARS'] = merged_data['UNIT DEMAND'] * merged_data['FULL_PRICE_RETAIL']

print(f"Merged data: {len(merged_data):,} rows")
print(f"\nSample merged data:")
merged_data.head(10)

Merged data: 146,616 rows

Sample merged data:


,SKU,MONTH,UNIT DEMAND,UNIT SALES,YEAR,MONTH_NAME,YEAR_MONTH,CATEGORY,COLLECTION,FULL_PRICE_RETAIL,RETAIL_DEMAND_DOLLARS
0,000004556-004-FL-S-002-B1,2025-01-01,0.0,0.0,2025,Jan,2025-01,FURNITURE,DRIFT,0.0,0.0
1,000004556-004-FL-S-002-B1,2025-02-01,0.0,0.0,2025,Feb,2025-02,FURNITURE,DRIFT,0.0,0.0
2,000004556-004-FL-S-002-B1,2025-03-01,0.0,0.0,2025,Mar,2025-03,FURNITURE,DRIFT,0.0,0.0
3,000004556-004-FL-S-002-B1,2025-04-01,0.0,0.0,2025,Apr,2025-04,FURNITURE,DRIFT,0.0,0.0
4,000004556-004-FL-S-002-B1,2025-05-01,0.0,0.0,2025,May,2025-05,FURNITURE,DRIFT,0.0,0.0
5,000004556-004-FL-S-002-B1,2025-06-01,0.0,0.0,2025,Jun,2025-06,FURNITURE,DRIFT,0.0,0.0
6,000004556-004-FL-S-002-B1,2025-07-01,0.0,0.0,2025,Jul,2025-07,FURNITURE,DRIFT,0.0,0.0
7,000004556-004-FL-S-002-B1,2025-08-01,0.0,0.0,2025,Aug,2025-08,FURNITURE,DRIFT,0.0,0.0
8,000004556-004-FL-S-002-B1,2025-09-01,0.0,0.0,2025,Sep,2025-09,FURNITURE,DRIFT,0.0,0.0
9,000004556-004-FL-S-002-B1,2025-10-01,0.0,0.0,2025,Oct,2025-10,FURNITURE,DRIFT,0.0,0.0


## Summary by Category

In [5]:
def create_pivot_with_totals(data, index_col, values_col='RETAIL_DEMAND_DOLLARS'):
    """
    Create a pivot table with monthly columns, year subtotals, and grand total.
    Returns an HTML table with collapsible year sections.
    """
    # Create pivot table
    pivot = data.pivot_table(
        index=index_col,
        columns=['YEAR', 'MONTH_NAME'],
        values=values_col,
        aggfunc='sum',
        fill_value=0
    )
    
    # Reorder months
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    # Create ordered columns
    ordered_cols = []
    for year in [2025, 2026]:
        for month in month_order:
            if (year, month) in pivot.columns:
                ordered_cols.append((year, month))
    
    # Reorder pivot columns
    pivot = pivot.reindex(columns=ordered_cols, fill_value=0)
    
    # Calculate year totals
    for year in [2025, 2026]:
        year_cols = [col for col in pivot.columns if col[0] == year]
        if year_cols:
            pivot[(year, 'TOTAL')] = pivot[year_cols].sum(axis=1)
    
    # Calculate grand total
    pivot[('GRAND', 'TOTAL')] = pivot.sum(axis=1)
    
    # Add row totals
    totals_row = pivot.sum(axis=0)
    totals_row.name = 'TOTAL'
    pivot = pd.concat([pivot, totals_row.to_frame().T])
    
    return pivot

# Create Category summary
category_pivot = create_pivot_with_totals(merged_data, 'CATEGORY')
print("Summary by Category - Retail Demand Dollars")
print("="*100)

Summary by Category - Retail Demand Dollars


In [6]:
def format_pivot_to_html(pivot_df, title):
    """
    Format pivot table as HTML with collapsible year sections and proper formatting.
    """
    # Format numbers as currency
    formatted_df = pivot_df.copy()
    for col in formatted_df.columns:
        formatted_df[col] = formatted_df[col].apply(lambda x: f"${x:,.0f}" if pd.notna(x) else "$0")
    
    # Generate unique ID for this table
    import hashlib
    table_id = hashlib.md5(title.encode()).hexdigest()[:8]
    
    # Build HTML with collapsible sections
    html = f"""
    <style>
        .summary-table {{
            border-collapse: collapse;
            font-family: Arial, sans-serif;
            font-size: 12px;
            width: 100%;
        }}
        .summary-table th, .summary-table td {{
            border: 1px solid #ddd;
            padding: 8px;
            text-align: right;
            white-space: nowrap;
        }}
        .summary-table th {{
            background-color: #4472C4;
            color: white;
            font-weight: bold;
        }}
        .summary-table th.year-header {{
            background-color: #2F5496;
            cursor: pointer;
        }}
        .summary-table th.year-header:hover {{
            background-color: #1F3864;
        }}
        .summary-table td:first-child, .summary-table th:first-child {{
            text-align: left;
            min-width: 150px;
        }}
        .summary-table tr:nth-child(even) {{
            background-color: #f2f2f2;
        }}
        .summary-table tr:hover {{
            background-color: #ddd;
        }}
        .summary-table tr.total-row {{
            background-color: #FFC000 !important;
            font-weight: bold;
        }}
        .summary-table td.year-total, .summary-table th.year-total {{
            background-color: #E2EFDA;
            font-weight: bold;
        }}
        .summary-table td.grand-total, .summary-table th.grand-total {{
            background-color: #FFC000;
            font-weight: bold;
        }}
        .collapsible {{
            display: table-cell;
        }}
        .collapsed {{
            display: none;
        }}
        h3 {{
            color: #2F5496;
            margin-top: 20px;
        }}
    </style>
    
    <h3>{title}</h3>
    <p><em>Click on year headers (2025/2026) to collapse/expand months</em></p>
    <table class="summary-table" id="table_{table_id}">
    """
    
    # Build header rows
    html += "<thead>\n<tr>\n"
    html += f'<th rowspan="2">{formatted_df.index.name or "Category"}</th>\n'
    
    # Year header row
    current_year = None
    year_spans = {}
    for col in formatted_df.columns:
        year = col[0]
        if year not in year_spans:
            year_spans[year] = 0
        year_spans[year] += 1
    
    for year, span in year_spans.items():
        if year == 'GRAND':
            html += f'<th colspan="{span}" class="grand-total">GRAND</th>\n'
        else:
            html += f'<th colspan="{span}" class="year-header" onclick="toggleYear{table_id}({year})">{year} (click to collapse)</th>\n'
    html += "</tr>\n<tr>\n"
    
    # Month header row
    for col in formatted_df.columns:
        year, month = col
        if month == 'TOTAL' and year != 'GRAND':
            html += f'<th class="year-total year-{year}-total">{year} Total</th>\n'
        elif year == 'GRAND':
            html += f'<th class="grand-total">Grand Total</th>\n'
        else:
            html += f'<th class="collapsible year-{year}">{month}</th>\n'
    html += "</tr>\n</thead>\n<tbody>\n"
    
    # Data rows
    for idx, row in formatted_df.iterrows():
        row_class = 'total-row' if idx == 'TOTAL' else ''
        html += f'<tr class="{row_class}">\n'
        html += f'<td>{idx}</td>\n'
        for col in formatted_df.columns:
            year, month = col
            value = row[col]
            if month == 'TOTAL' and year != 'GRAND':
                html += f'<td class="year-total year-{year}-total">{value}</td>\n'
            elif year == 'GRAND':
                html += f'<td class="grand-total">{value}</td>\n'
            else:
                html += f'<td class="collapsible year-{year}">{value}</td>\n'
        html += "</tr>\n"
    
    html += "</tbody>\n</table>\n"
    
    # JavaScript for collapsing
    html += f"""
    <script>
    function toggleYear{table_id}(year) {{
        var cells = document.querySelectorAll('#table_{table_id} .year-' + year);
        cells.forEach(function(cell) {{
            if (cell.classList.contains('collapsed')) {{
                cell.classList.remove('collapsed');
                cell.classList.add('collapsible');
            }} else {{
                cell.classList.remove('collapsible');
                cell.classList.add('collapsed');
            }}
        }});
    }}
    </script>
    """
    
    return html

# Display Category summary
category_html = format_pivot_to_html(category_pivot, "Retail Demand Dollars by Category")
display(HTML(category_html))

## Summary by Collection

In [7]:
# Create Collection summary
collection_pivot = create_pivot_with_totals(merged_data, 'COLLECTION')

# Display Collection summary
collection_html = format_pivot_to_html(collection_pivot, "Retail Demand Dollars by Collection")
display(HTML(collection_html))

## Summary by Category and Collection

In [8]:
def create_hierarchical_pivot(data, values_col='RETAIL_DEMAND_DOLLARS'):
    """
    Create a hierarchical pivot table with Category and Collection.
    """
    # Create pivot table with multi-index
    pivot = data.pivot_table(
        index=['CATEGORY', 'COLLECTION'],
        columns=['YEAR', 'MONTH_NAME'],
        values=values_col,
        aggfunc='sum',
        fill_value=0
    )
    
    # Reorder months
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    # Create ordered columns
    ordered_cols = []
    for year in [2025, 2026]:
        for month in month_order:
            if (year, month) in pivot.columns:
                ordered_cols.append((year, month))
    
    # Reorder pivot columns
    pivot = pivot.reindex(columns=ordered_cols, fill_value=0)
    
    # Calculate year totals
    for year in [2025, 2026]:
        year_cols = [col for col in pivot.columns if col[0] == year]
        if year_cols:
            pivot[(year, 'TOTAL')] = pivot[year_cols].sum(axis=1)
    
    # Calculate grand total
    pivot[('GRAND', 'TOTAL')] = pivot.sum(axis=1)
    
    # Add row totals
    totals_row = pivot.sum(axis=0)
    totals_row.name = ('TOTAL', 'TOTAL')
    pivot = pd.concat([pivot, totals_row.to_frame().T])
    
    return pivot

# Create hierarchical summary
hierarchical_pivot = create_hierarchical_pivot(merged_data)
print(f"Hierarchical pivot created: {len(hierarchical_pivot)} rows")

Hierarchical pivot created: 678 rows


In [9]:
def format_hierarchical_pivot_to_html(pivot_df, title):
    """
    Format hierarchical pivot table as HTML with collapsible year sections.
    """
    # Format numbers as currency
    formatted_df = pivot_df.copy()
    for col in formatted_df.columns:
        formatted_df[col] = formatted_df[col].apply(lambda x: f"${x:,.0f}" if pd.notna(x) else "$0")
    
    # Generate unique ID for this table
    import hashlib
    table_id = hashlib.md5(title.encode()).hexdigest()[:8]
    
    # Build HTML
    html = f"""
    <style>
        .hier-table {{
            border-collapse: collapse;
            font-family: Arial, sans-serif;
            font-size: 11px;
            width: 100%;
        }}
        .hier-table th, .hier-table td {{
            border: 1px solid #ddd;
            padding: 6px;
            text-align: right;
            white-space: nowrap;
        }}
        .hier-table th {{
            background-color: #4472C4;
            color: white;
            font-weight: bold;
        }}
        .hier-table th.year-header {{
            background-color: #2F5496;
            cursor: pointer;
        }}
        .hier-table th.year-header:hover {{
            background-color: #1F3864;
        }}
        .hier-table td.category, .hier-table td.collection {{
            text-align: left;
            min-width: 120px;
        }}
        .hier-table th.category-header, .hier-table th.collection-header {{
            text-align: left;
            min-width: 120px;
        }}
        .hier-table tr:nth-child(even) {{
            background-color: #f9f9f9;
        }}
        .hier-table tr:hover {{
            background-color: #eee;
        }}
        .hier-table tr.total-row {{
            background-color: #FFC000 !important;
            font-weight: bold;
        }}
        .hier-table td.year-total {{
            background-color: #E2EFDA;
            font-weight: bold;
        }}
        .hier-table td.grand-total {{
            background-color: #FFC000;
            font-weight: bold;
        }}
        .hier-table th.grand-total {{
            background-color: #FFC000;
            color: black;
        }}
        .collapsible {{
            display: table-cell;
        }}
        .collapsed {{
            display: none;
        }}
        h3 {{
            color: #2F5496;
            margin-top: 30px;
        }}
    </style>
    
    <h3>{title}</h3>
    <p><em>Click on year headers (2025/2026) to collapse/expand months</em></p>
    <table class="hier-table" id="table_{table_id}">
    """
    
    # Build header rows
    html += "<thead>\n<tr>\n"
    html += '<th rowspan="2" class="category-header">Category</th>\n'
    html += '<th rowspan="2" class="collection-header">Collection</th>\n'
    
    # Year header row
    year_spans = {}
    for col in formatted_df.columns:
        year = col[0]
        if year not in year_spans:
            year_spans[year] = 0
        year_spans[year] += 1
    
    for year, span in year_spans.items():
        if year == 'GRAND':
            html += f'<th colspan="{span}" class="grand-total">GRAND</th>\n'
        else:
            html += f'<th colspan="{span}" class="year-header" onclick="toggleYearHier{table_id}({year})">{year} (click to collapse)</th>\n'
    html += "</tr>\n<tr>\n"
    
    # Month header row
    for col in formatted_df.columns:
        year, month = col
        if month == 'TOTAL' and year != 'GRAND':
            html += f'<th class="year-total year-{year}-total">{year} Total</th>\n'
        elif year == 'GRAND':
            html += f'<th class="grand-total">Grand Total</th>\n'
        else:
            html += f'<th class="collapsible year-{year}">{month}</th>\n'
    html += "</tr>\n</thead>\n<tbody>\n"
    
    # Data rows
    for idx, row in formatted_df.iterrows():
        if isinstance(idx, tuple):
            cat, coll = idx
        else:
            cat, coll = idx, ''
        
        row_class = 'total-row' if cat == 'TOTAL' else ''
        html += f'<tr class="{row_class}">\n'
        html += f'<td class="category">{cat}</td>\n'
        html += f'<td class="collection">{coll}</td>\n'
        
        for col in formatted_df.columns:
            year, month = col
            value = row[col]
            if month == 'TOTAL' and year != 'GRAND':
                html += f'<td class="year-total year-{year}-total">{value}</td>\n'
            elif year == 'GRAND':
                html += f'<td class="grand-total">{value}</td>\n'
            else:
                html += f'<td class="collapsible year-{year}">{value}</td>\n'
        html += "</tr>\n"
    
    html += "</tbody>\n</table>\n"
    
    # JavaScript for collapsing
    html += f"""
    <script>
    function toggleYearHier{table_id}(year) {{
        var cells = document.querySelectorAll('#table_{table_id} .year-' + year);
        cells.forEach(function(cell) {{
            if (cell.classList.contains('collapsed')) {{
                cell.classList.remove('collapsed');
                cell.classList.add('collapsible');
            }} else {{
                cell.classList.remove('collapsible');
                cell.classList.add('collapsed');
            }}
        }});
    }}
    </script>
    """
    
    return html

# Display hierarchical summary
hierarchical_html = format_hierarchical_pivot_to_html(hierarchical_pivot, "Retail Demand Dollars by Category and Collection")
display(HTML(hierarchical_html))

## Export to Excel

In [10]:
# Export all summaries to Excel with formatting
output_excel_file = os.path.join(data_path, 'Summary_Forecast_and_Receipts.xlsx')

with pd.ExcelWriter(output_excel_file, engine='xlsxwriter') as writer:
    workbook = writer.book
    
    # Define formats
    header_format = workbook.add_format({
        'bold': True,
        'bg_color': '#4472C4',
        'font_color': 'white',
        'border': 1,
        'align': 'center',
        'valign': 'vcenter',
        'text_wrap': True
    })
    
    year_total_format = workbook.add_format({
        'bold': True,
        'bg_color': '#E2EFDA',
        'border': 1,
        'num_format': '$#,##0',
        'align': 'right'
    })
    
    grand_total_format = workbook.add_format({
        'bold': True,
        'bg_color': '#FFC000',
        'border': 1,
        'num_format': '$#,##0',
        'align': 'right'
    })
    
    currency_format = workbook.add_format({
        'num_format': '$#,##0',
        'border': 1,
        'align': 'right'
    })
    
    text_format = workbook.add_format({
        'border': 1,
        'align': 'left'
    })
    
    total_row_format = workbook.add_format({
        'bold': True,
        'bg_color': '#FFC000',
        'border': 1,
        'num_format': '$#,##0',
        'align': 'right'
    })
    
    # Write Category summary
    category_pivot.to_excel(writer, sheet_name='By Category', startrow=1)
    worksheet1 = writer.sheets['By Category']
    worksheet1.set_column('A:A', 20)  # Category column
    worksheet1.set_column('B:Z', 12)  # Month columns
    worksheet1.freeze_panes(3, 1)  # Freeze headers and first column
    
    # Write Collection summary
    collection_pivot.to_excel(writer, sheet_name='By Collection', startrow=1)
    worksheet2 = writer.sheets['By Collection']
    worksheet2.set_column('A:A', 25)  # Collection column
    worksheet2.set_column('B:Z', 12)  # Month columns
    worksheet2.freeze_panes(3, 1)
    
    # Write Category + Collection summary
    hierarchical_pivot.to_excel(writer, sheet_name='By Category-Collection', startrow=1)
    worksheet3 = writer.sheets['By Category-Collection']
    worksheet3.set_column('A:A', 18)  # Category column
    worksheet3.set_column('B:B', 25)  # Collection column
    worksheet3.set_column('C:AZ', 12)  # Month columns
    worksheet3.freeze_panes(3, 2)  # Freeze headers and first two columns

print(f"\n✓ Excel file exported to: {output_excel_file}")
print(f"  Sheets: 'By Category', 'By Collection', 'By Category-Collection'")


✓ Excel file exported to: C:\Users\JaeheeKim\demand-forecast-hackathon-1\notebooks\..\data\Summary_Forecast_and_Receipts.xlsx
  Sheets: 'By Category', 'By Collection', 'By Category-Collection'


## Summary Statistics

In [11]:
# Display summary statistics
print("=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)

# Total demand by year
yearly_totals = merged_data.groupby('YEAR')['RETAIL_DEMAND_DOLLARS'].sum()
print(f"\nTotal Retail Demand Dollars:")
for year, total in yearly_totals.items():
    print(f"  {year}: ${total:,.0f}")
print(f"  Grand Total: ${yearly_totals.sum():,.0f}")

# Top 10 categories
print(f"\nTop 10 Categories by Retail Demand:")
top_categories = merged_data.groupby('CATEGORY')['RETAIL_DEMAND_DOLLARS'].sum().sort_values(ascending=False).head(10)
for i, (cat, total) in enumerate(top_categories.items(), 1):
    print(f"  {i}. {cat}: ${total:,.0f}")

# Top 10 collections
print(f"\nTop 10 Collections by Retail Demand:")
top_collections = merged_data.groupby('COLLECTION')['RETAIL_DEMAND_DOLLARS'].sum().sort_values(ascending=False).head(10)
for i, (coll, total) in enumerate(top_collections.items(), 1):
    print(f"  {i}. {coll}: ${total:,.0f}")

SUMMARY STATISTICS

Total Retail Demand Dollars:
  2025: $65,550,883
  2026: $65,550,883
  Grand Total: $131,101,766

Top 10 Categories by Retail Demand:
  1. RUGS: $39,597,711
  2. BEDDING: $32,475,313
  3. FURNITURE: $21,568,872
  4. PILLOWS: $14,751,176
  5. ACCENTS: $8,430,763
  6. BATH: $4,401,228
  7. BASKETS: $4,309,044
  8. BLANKETS: $4,009,751
  9. TABLEWARE: $1,502,654
  10. Z. MISCELLANEOUS: $55,254

Top 10 Collections by Retail Demand:
  1. STONEWASHED LINEN: $18,925,075
  2. DUNES: $7,003,620
  3. NAIRA: $4,613,476
  4. ORGANIC COTTON PERCALE: $4,162,423
  5. USTAV: $3,920,064
  6. HINOKI: $2,367,178
  7. NAVETA: $2,316,604
  8. COTTON GAUZE: $1,925,755
  9. ARTHA: $1,841,637
  10. SALINA: $1,747,365
